In [0]:
from keras.datasets import mnist
from keras.utils import to_categorical

from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from keras.models import Sequential
from keras.models import load_model

import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive/')

## データ読み込み

In [0]:
X = []
Y = []
image_size = 30

folder_path = "/content/drive/My Drive/PythonBooks/src/Ch07/images/"
file_list = os.listdir(folder_path)

In [0]:
for file in file_list:
    try:
        image = Image.open(folder_path + file)
    except:
        print('error', file)
        continue
    
    image = image.convert("RGB")
    image = image.resize((image_size, image_size))
    data = np.asarray(image)
    X.append(data)
    if 'temple' in file:
        Y.append(0)
    else:
         Y.append(1)

X = np.array(X)
Y = np.array(Y)

### データ整形

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)
# 画像をreshape
X_train = X_train.reshape(-1, image_size, image_size, 3)
X_test = X_test.reshape(-1, image_size, image_size, 3)

# 輝度値を0 ~ 1に入るように正規化
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

# one hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

## モデル作成

In [0]:
model_cnn = Sequential()
model_cnn.add(Conv2D(32, (3,3), activation='relu', input_shape=(image_size, image_size, 3)))
model_cnn.add(MaxPool2D(2,2))
model_cnn.add(Conv2D(64, (3,3), activation='relu'))
model_cnn.add(MaxPool2D(2,2))
model_cnn.add(Conv2D(128, (3,3), activation='relu'))
model_cnn.add(MaxPool2D(2,2))
model_cnn.add(Flatten())
model_cnn.add(Dense(512, activation='relu'))       
model_cnn.add(Dense(2, activation='softmax'))

model_cnn.summary()

In [0]:
model_cnn.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_cnn.fit(X_train, y_train, epochs=10, batch_size=20)

model_cnn.evaluate(X_test, y_test)